In [15]:
import math
import scipy
from scipy import special
import numpy as np
import pickle 
from scipy.stats import binom
import matplotlib.pyplot as plt
import scipy.stats
import csv
import time
import torch

In [16]:
clk_list=["4","6","8","10","12","16","20"]

In [17]:
def binom_func(n,p,k):
    val = binom.pmf(k, n, p)
    #val = scipy.special.binom(n,k)*(p**k)*((1-p)**(n-k))
    return val

def std_clk(clk_period, nsize):
    with open("./sigma_linear_fit_bias.csv", newline='') as f:
        reader = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC)
        coeff_list = list(reader)
    [a,b] = coeff_list[clk_list.index(clk_period)]
    return a*nsize+b

def SA_error_pdf_faster(delta, std_comp):
    delta_abs = np.abs(delta)+1 # because 0 is not a real curve
    return 1/2*np.exp(-delta_abs*delta_abs/(2*std_comp*std_comp))

def prob_fail_delta_total_faster(nc, delta, p, std_comp=0, with_SA=None):
    probi_SA_correct=0
    probi_SA_incorrect=0
    
    if delta<0:
        print("Oulala, delta doit être supérieur ou égal à 0... !")
        return
        
    nc_half_up = int(math.ceil(nc/2))
    nc_half_down = int(math.floor(nc/2))
    
    
    ###
    binom_vals_j = binom_func(nc_half_down-delta,p,range(0, nc_half_down-delta+1))
    binom_vals_i = binom_func(nc_half_up+delta,p,range(0, nc_half_up+delta+1))
    ###
    
    for i in range(1+delta, nc_half_up+delta+1):
        probj=0
        j = np.arange(0, min(nc_half_down-delta, i-1-delta)+1, 1)
        if with_SA==False:
            proba_SA_erreur = 0
        else :
            proba_SA_erreur = SA_error_pdf_faster(nc_half_down-(i+nc_half_down-delta-j), std_comp)
        proba_SA_equal_to_one = (1-proba_SA_erreur)
        
        probj = binom_vals_j[0:min(nc_half_down-delta, i-1-delta)+1]@proba_SA_equal_to_one
            
        probi_SA_correct += binom_vals_i[i]*probj

    for i in range(0, nc_half_down+1):
        probj=0
        j = np.arange(max(0, i-delta), nc_half_down-delta+1, 1)
        if with_SA==False:
            proba_SA_erreur = 0
        else :
            proba_SA_erreur = SA_error_pdf_faster(nc_half_down-(i+nc_half_down-delta-j),std_comp)
        proba_SA_equal_to_one = proba_SA_erreur
        probj = binom_vals_j[max(0, i-delta):nc_half_down-delta+1]@proba_SA_equal_to_one
            
        probi_SA_incorrect += binom_vals_i[i]*probj
        
    probi = probi_SA_correct + probi_SA_incorrect
    return probi

def proba_list_faster(nc, nfilter, list_p, clk_period="8", with_SA=None):
    list_global = []
    nc_inputs = nc*nfilter
    nc_bias = nc_inputs+2*int(0.05*nc_inputs)
    
    std_comp = std_clk(clk_period, nc_bias)
    
    nc_half_up = int(math.ceil(nc_bias/2))
    nc_half_down = int(math.floor(nc_bias/2))
    
    for pi in list_p:
        print("start "+str(pi))
        proba_up = []
        single_err_for_p_half = prob_fail_delta_total_faster(nc_bias, 0, 0.5, std_comp, with_SA=True)
        for delta in range(0, nc_half_down+1):
            
            if pi==0.5:
                proba_up.append(single_err_for_p_half)
                
            else:            
                if delta>0 and proba_up[delta-1]<1e-10:
                    proba_up.append(0.0)
                else:
                    proba_up.append(prob_fail_delta_total_faster(nc_bias, delta, pi, std_comp, with_SA))
        list_global.append(proba_up)
    return np.array(list_global)

def gauss(x, sigma):
    out = 0.5*torch.exp(-0.5*(x/sigma)**2)
    return out

In [18]:
clock = "6"

list_p_1 = [0, 0.0001, 0.001, 0.002, 0.003, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5]

list_p_2 = (np.array([0.072464, 0.036232, 0.028986, 0.007246, 0.144928, 0.021739, 0.028986, 0.014493, 0.927536, 0.282609, 0.260870, 0.268116, 7.811594, 2.702899, 2.289855, 2.376812, 2.485507])/100).tolist()

list_p = list_p_1+list_p_2

t = time.time()
proba_array_25137_wSA = proba_list_faster(nc=25137, nfilter=1, list_p=list_p, clk_period=clock, with_SA=True)

elapsed = time.time() - t
print(elapsed)

start 0
start 0.0001
start 0.001
start 0.002
start 0.003
start 0.01
start 0.02
start 0.03
start 0.04
start 0.05
start 0.06
start 0.07
start 0.08
start 0.09
start 0.1
start 0.2
start 0.3
start 0.4
start 0.5
start 0.00072464
start 0.00036232
start 0.00028986
start 7.246e-05
start 0.00144928
start 0.00021739
start 0.00028986
start 0.00014493
start 0.00927536
start 0.00282609
start 0.0026087
start 0.00268116
start 0.07811594000000001
start 0.02702899
start 0.022898550000000004
start 0.02376812
start 0.02485507
71453.94476437569


In [19]:
proba_array_3465_wSA = proba_list_faster(nc=3465, nfilter=1, list_p=list_p, clk_period=clock, with_SA=True)
print("End of 3465")

proba_array_6921_wSA = proba_list_faster(nc=6921, nfilter=1, list_p=list_p, clk_period=clock, with_SA=True)
print("End of 6921")

proba_array_13833_wSA = proba_list_faster(nc=13833, nfilter=1, list_p=list_p, clk_period=clock, with_SA=True)
print("End of 13833")

proba_array_4617_wSA = proba_list_faster(nc=4617, nfilter=1, list_p=list_p, clk_period=clock, with_SA=True)
print("End of 4617")

proba_array_1025_wSA = proba_list_faster(nc=1025, nfilter=1, list_p=list_p, clk_period=clock, with_SA=True)
print("End of 1025")

start 0
start 0.0001
start 0.001
start 0.002
start 0.003
start 0.01
start 0.02
start 0.03
start 0.04
start 0.05
start 0.06
start 0.07
start 0.08
start 0.09
start 0.1
start 0.2
start 0.3
start 0.4
start 0.5
start 0.00072464
start 0.00036232
start 0.00028986
start 7.246e-05
start 0.00144928
start 0.00021739
start 0.00028986
start 0.00014493
start 0.00927536
start 0.00282609
start 0.0026087
start 0.00268116
start 0.07811594000000001
start 0.02702899
start 0.022898550000000004
start 0.02376812
start 0.02485507
End of 3465
start 0
start 0.0001
start 0.001
start 0.002
start 0.003
start 0.01
start 0.02
start 0.03
start 0.04
start 0.05
start 0.06
start 0.07
start 0.08
start 0.09
start 0.1
start 0.2
start 0.3
start 0.4
start 0.5
start 0.00072464
start 0.00036232
start 0.00028986
start 7.246e-05
start 0.00144928
start 0.00021739
start 0.00028986
start 0.00014493
start 0.00927536
start 0.00282609
start 0.0026087
start 0.00268116
start 0.07811594000000001
start 0.02702899
start 0.02289855000000000

In [20]:
np.savetxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_25137.txt', proba_array_25137_wSA)
np.savetxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_3465.txt', proba_array_3465_wSA)
np.savetxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_6921.txt', proba_array_6921_wSA)
np.savetxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_13833.txt', proba_array_13833_wSA)
np.savetxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_4617.txt', proba_array_4617_wSA)
np.savetxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_1025.txt', proba_array_1025_wSA)

In [21]:
proba_array_1025_wSA.shape

(36, 564)

# New approach from loaded data

In [22]:
prob_array_half_n_25137_wSA = np.loadtxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_25137.txt')
prob_array_half_n_3465_wSA = np.loadtxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_3465.txt')
prob_array_half_n_6921_wSA = np.loadtxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_6921.txt')
prob_array_half_n_13833_wSA = np.loadtxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_13833.txt')
prob_array_half_n_4617_wSA = np.loadtxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_4617.txt')
prob_array_half_n_1025_wSA = np.loadtxt(f'./error_all/{clock}_ns/One_sided_error/One_sided_error_prob_with_SA_clk_{clock}_ns_neurons_1025.txt')


prob_array_full_n_25137_wSA = np.zeros([13826, 36])
prob_array_full_n_3465_wSA = np.zeros([1907, 36])
prob_array_full_n_6921_wSA = np.zeros([3808, 36])
prob_array_full_n_13833_wSA = np.zeros([7609, 36])
prob_array_full_n_4617_wSA = np.zeros([2540, 36])
prob_array_full_n_1025_wSA = np.zeros([565, 36])


prob_array_full_n_1025_wSA[1:, :] = prob_array_half_n_1025_wSA.transpose()
max_1025_wSA = np.max(prob_array_half_n_1025_wSA, axis=-1)
prob_array_full_n_1025_wSA[0, :] = np.where(max_1025_wSA<=0.5, 0.5, max_1025_wSA)

prob_array_full_n_4617_wSA[1:, :] = prob_array_half_n_4617_wSA.transpose()
max_4617_wSA = np.max(prob_array_half_n_4617_wSA, axis=-1)
prob_array_full_n_4617_wSA[0, :] = np.where(max_4617_wSA<=0.5, 0.5, max_4617_wSA)

prob_array_full_n_13833_wSA[1:, :] = prob_array_half_n_13833_wSA.transpose()
max_13833_wSA = np.max(prob_array_half_n_13833_wSA, axis=-1)
prob_array_full_n_13833_wSA[0, :] = np.where(max_13833_wSA<=0.5, 0.5, max_13833_wSA)

prob_array_full_n_6921_wSA[1:, :] = prob_array_half_n_6921_wSA.transpose()
max_6921_wSA = np.max(prob_array_half_n_6921_wSA, axis=-1)
prob_array_full_n_6921_wSA[0, :] = np.where(max_6921_wSA<=0.5, 0.5, max_6921_wSA)

prob_array_full_n_3465_wSA[1:, :] = prob_array_half_n_3465_wSA.transpose()
max_3465_wSA = np.max(prob_array_half_n_3465_wSA, axis=-1)
prob_array_full_n_3465_wSA[0, :] = np.where(max_3465_wSA<=0.5, 0.5, max_3465_wSA)

prob_array_full_n_25137_wSA[1:, :] = prob_array_half_n_25137_wSA.transpose()
max_25137_wSA = np.max(prob_array_half_n_25137_wSA, axis=-1)
prob_array_full_n_25137_wSA[0, :] = np.where(max_25137_wSA<=0.5, 0.5, max_25137_wSA)

In [23]:
np.savetxt(f'./error_all/{clock}_ns/Full_error_prob_with_SA_clk_{clock}_ns_neurons_25137.txt', prob_array_full_n_25137_wSA)
np.savetxt(f'./error_all/{clock}_ns/Full_error_prob_with_SA_clk_{clock}_ns_neurons_3465.txt', prob_array_full_n_3465_wSA)
np.savetxt(f'./error_all/{clock}_ns/Full_error_prob_with_SA_clk_{clock}_ns_neurons_6921.txt', prob_array_full_n_6921_wSA)
np.savetxt(f'./error_all/{clock}_ns/Full_error_prob_with_SA_clk_{clock}_ns_neurons_13833.txt', prob_array_full_n_13833_wSA)
np.savetxt(f'./error_all/{clock}_ns/Full_error_prob_with_SA_clk_{clock}_ns_neurons_4617.txt', prob_array_full_n_4617_wSA)
np.savetxt(f'./error_all/{clock}_ns/Full_error_prob_with_SA_clk_{clock}_ns_neurons_1025.txt', prob_array_full_n_1025_wSA)